In [ ]:
from FuncionAzul import *

In [ ]:
import os

In [ ]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.3.0 pyspark-shell'

In [1]:
from function2 import *
import pickle
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.window import Window as W

# Prueba 

In [2]:
FuncionAzul(Ruta_Nombre_Input_entidades = "DATA_SIC_ENTIDADES_PUBLICAS_SOLICITUD_ESP.csv", 
            Ruta_Nombre_Input_investigadores = "DATA_SIC_INVEST_UO_FIL_PB_J_MAESTRO.csv",     
            Ruta_Nombre_Input_relaciones = "DATA_SIC_RELACIONES_ENTIDADES_PB_AYUDAS_ESP_A6CORREC_new.csv",
            Ruta_Nombre_Input_invente = 'Analisis_entidades_invente_15-02-2022.xlsx', 
            Ruta_Nombre_Input_dir3 = "DIR3_concatenado_AGE_CCAA_EELL_Universidades.csv", 
            Ruta_Nombre_Intermedio_cruce1_relaciones = "test/Nombre_Intermedio_cruce1_relaciones",
            Ruta_Nombre_Intermedio_cruce1_entidades = "test/Nombre_Intermedio_cruce1_entidades", 
            Ruta_Nombre_Intermedio_cruce2_invente = "test/Nombre_Intermedio_cruce2_invente", 
            Ruta_Nombre_Intermedio_cruce3_dir3 = "test/Nombre_Intermedio_cruce3_dir3",
            Ruta_Nombre_Output_investigadores = "test/Nombre_Output_investigadores", 
            Ruta_Nombre_Output_relaciones = "test/Nombre_Output_relaciones",
            Ruta_Nombre_Output_entidades = "test/Nombre_Output_entidades", 
            flag_invente_dir3 = True, 
            Flag_csv = True)

NameError: name 'FuncionAzul' is not defined

In [2]:
Ruta_Nombre_Input_entidades = "DATA_SIC_ENTIDADES_PUBLICAS_SOLICITUD_ESP.csv" 
Ruta_Nombre_Input_investigadores = "DATA_SIC_INVEST_UO_FIL_PB_J_MAESTRO.csv"     
Ruta_Nombre_Input_relaciones = "DATA_SIC_RELACIONES_ENTIDADES_PB_AYUDAS_ESP_A6CORREC_new.csv"
Ruta_Nombre_Input_invente = 'Analisis_entidades_invente_15-02-2022.xlsx' 
Ruta_Nombre_Input_dir3 = "DIR3_concatenado_AGE_CCAA_EELL_Universidades.csv" 
Ruta_Nombre_Intermedio_cruce1_relaciones = "test/Nombre_Intermedio_cruce1_relaciones"
Ruta_Nombre_Intermedio_cruce1_entidades = "test/Nombre_Intermedio_cruce1_entidades" 
Ruta_Nombre_Intermedio_cruce2_invente = "test/Nombre_Intermedio_cruce2_invente" 
Ruta_Nombre_Intermedio_cruce3_dir3 = "test/Nombre_Intermedio_cruce3_dir3"
Ruta_Nombre_Output_investigadores = "test/Nombre_Output_investigadores" 
Ruta_Nombre_Output_relaciones = "test/Nombre_Output_relaciones"
Ruta_Nombre_Output_entidades = "test/Nombre_Output_entidades" 
flag_invente_dir3 = True 
Flag_csv = True

In [3]:
spark = SparkSession.builder.appName('FuncionAzul').config("spark.driver.memory","70G")\
                         .config("spark.executor.memory","100G")\
                         .config("spark.executor.cores","20")\
                         .config("spark.executor.instances","5")\
                         .config("spark.driver.maxResultSize", '128g')\
                         .config("spark.memory.offHeap.enabled", 'true')\
                         .config("spark.memory.offHeap.size", '30g')\
                         .enableHiveSupport().getOrCreate()

In [4]:
# =============================================================================
# Creamos 3 archivos: 
# 1. Nombre_Intermedio_cruce1_entidades.
# 2. Nombre_Output_investigadores.  
# 3. Nombre_Intermedio_cruce1_relaciones.
# =============================================================================
Nombre_Input_entidades = spark.read.csv(Ruta_Nombre_Input_entidades, sep=";", header=True)
Nombre_Input_relaciones = spark.read.csv(Ruta_Nombre_Input_relaciones, sep=";", header=True)
Nombre_Input_relaciones = Nombre_Input_relaciones.drop("Entidad_NIVEL_0", "Entidad_Norm_NIVEL_0", "CIF_NIVEL_0", "Provincia_Entidad_NIVEL_0", "Entidad_NIVEL_1",
                                                      "Entidad_Norm_NIVEL_1", "CIF_NIVEL_1", "Provincia_Entidad_NIVEL_1")
Nombre_Input_investigadores = spark.read.csv(Ruta_Nombre_Input_investigadores, sep=";", header=True)



cruce1 = Nombre_Input_investigadores.\
                                    join(Nombre_Input_relaciones,
                                         Nombre_Input_investigadores.Id ==  Nombre_Input_relaciones.Id_NIVEL_0,"left")\
                                    .dropDuplicates(Nombre_Input_investigadores.columns)\
                                    .filter("Id_NIVEL_0 is not null")



cruce1 = cruce1.withColumn("Similitud", 
                           udf_Distance_ratcliff_obershelp(
                               UDF_normalizarTexto(cruce1.NOMBRE_ENTIDAD_NIVEL_1),
                               cruce1.Centro_Norm )
                          ) 


# # Nombre_Intermedio_cruce1_entidades

# Valor maximo del index del archivo Nombre_Input_entidades
maxId = Nombre_Input_entidades.withColumn("Id", Nombre_Input_entidades.Id.cast('int')).select("Id").rdd.max()[0]




data = cruce1.filter("Similitud <0.875")\
             .withColumn("CIF",lit(None).cast(StringType()))\
             .withColumn("Entidad_Norm",col("Centro_Norm"))\
             .withColumn("Provincia_Entidad",col("Provincia_Centro"))\
             .withColumn("ID_ENTIDAD",lit(None).cast(StringType()))\
             .withColumn("NIF_COD",lit(None).cast(StringType()))\
             .withColumn("ACRONIMO",lit(None).cast(StringType()))\
             .withColumn("NOMBRE_ENTIDAD",lit(None).cast(StringType()))\
             .withColumn("Nombre_Entidad_Mostrar",lit(None).cast(StringType()))\
             .withColumn("TIPO_ENTIDAD_N1_1",lit(None).cast(StringType()))\
             .withColumn("TIPO_ENTIDAD_N2_1",lit(None).cast(StringType()))\
             .withColumn("DIRECCION_POSTAL",lit(None).cast(StringType()))\
             .withColumn("COD_POSTAL",lit(None).cast(StringType()))\
             .withColumn("COD_PROVINCIA",lit(None).cast(StringType()))\
             .withColumn("PROVINCIA",lit(None).cast(StringType()))\
             .withColumn("COD_CCAA",lit(None).cast(StringType()))\
             .withColumn("CCAA",lit(None).cast(StringType()))\
             .withColumn("ENLACE_WEB",lit(None).cast(StringType()))\
             .withColumn("SOMMA",lit(None).cast(StringType()))\
             .withColumn("TIPO_ENTIDAD_REGIONAL",lit(None).cast(StringType()))\
             .withColumn("ESTADO_x",lit(None).cast(StringType()))\
             .withColumn("CodigoInvente",lit(None).cast(StringType()))\
             .withColumn("DenominacionSocial",lit(None).cast(StringType()))\
             .withColumn("FormaJuridica_Codigo",lit(None).cast(StringType()))\
             .withColumn("FormaJuridica_Descripcion",lit(None).cast(StringType()))\
             .withColumn("NIF",lit(None).cast(StringType()))\
             .withColumn("codigoDir3",lit(None).cast(StringType()))\
             .withColumn("codigoOrigen",lit(None).cast(StringType()))\
             .withColumn("Provincia_Codigo",lit(None).cast(StringType()))\
             .withColumn("C_ID_UD_ORGANICA",lit(None).cast(StringType()))\
             .withColumn("C_DNM_UD_ORGANICA",lit(None).cast(StringType()))\
             .withColumn("C_ID_NIVEL_ADMON",lit(None).cast(StringType()))\
             .withColumn("C_ID_TIPO_ENT_PUBLICA",lit(None).cast(StringType()))\
             .withColumn("N_NIVEL_JERARQUICO",lit(None).cast(StringType()))\
             .withColumn("C_ID_DEP_UD_SUPERIOR",lit(None).cast(StringType()))\
             .withColumn("C_DNM_UD_ORGANICA_SUPERIOR",lit(None).cast(StringType()))\
             .withColumn("C_ID_DEP_UD_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("C_DNM_UD_ORGANICA_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("B_SW_DEP_EDP_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("C_ID_DEP_EDP_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("C_DNM_UD_ORGANICA_EDP_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("C_ID_ESTADO",lit(None).cast(StringType()))\
             .withColumn("D_VIG_ALTA_OFICIAL",lit(None).cast(StringType()))\
             .withColumn("NIF_CIF",lit(None).cast(StringType()))\
             .withColumn("C_ID_AMB_PROVINCIA",lit(None).cast(StringType()))\
             .withColumn("C_DESC_PROV",lit(None).cast(StringType()))\
             .withColumn("CONTACTOS",lit(None).cast(StringType()))\
             .withColumn("List_Entidad_Norm",col("Centro_Norm"))\
             .withColumn("List_Provincia_Entidad",col("Provincia_Centro"))\
             .withColumn("List_CIF",lit(None).cast(StringType()))



In [5]:
data_count = data.count()

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 41322)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/gaszsantana/virtual/lib/python3.8/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/gaszsantana/virtual/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/gaszsantana/virtual/lib/python3.8/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
Traceback (most recent call last):
  File "/usr/lib/python3.8/socketserver.py", line 316, in _handle_r

Py4JError: An error occurred while calling o536.count

In [ ]:
data_count

In [6]:


# =============================================================================
# Procesamiento para iniciar el archivo deseado a partter del maxId
# =============================================================================
data = data.dropDuplicates(["Entidad_Norm", "Provincia_Entidad"])
data = data.withColumnRenamed("Id", "IdOld")

Index = [maxId + i for i in range(1, data_count+1)]
Index = spark.createDataFrame(Index, IntegerType())
Index = Index.withColumnRenamed("value", "Id")




windowSpec = W.orderBy("Dummy")

Index = Index.withColumn("Dummy", monotonically_increasing_id())
Index = Index.withColumn("Dummy", F.row_number().over(windowSpec))

data = data.withColumn("Dummy", monotonically_increasing_id())
data = data.withColumn("Dummy", F.row_number().over(windowSpec))

data = data.join(Index, "Dummy", "outer").drop("Dummy")



Nombre_Intermedio_cruce1_entidades = data.select(Nombre_Input_entidades.columns)





# =============================================================================
# Aplicación de metodo para almacenar el resultado (parquet, csv, o ambos inclusive)
# =============================================================================
save_csv_parquet(Nombre_Intermedio_cruce1_entidades, Flag_csv, Ruta_Nombre_Intermedio_cruce1_entidades)


# # Creación del archivo Nombre_Output_investigadores


data_2 = cruce1.filter("Similitud >=0.875")\
        .withColumn("Final_Score", col("Similitud"))\
        .withColumn("Id_Centro", col("Id_NIVEL_1"))\
        .withColumn("Centro_Nombre_match", col("NOMBRE_ENTIDAD_NIVEL_1"))\
        .withColumn("Centro_Provincia_match", col("COD_PROVINCIA_NIVEL_1"))\
        .withColumn("Similitud_Centro", col("Similitud"))\
        .select(Nombre_Input_investigadores.columns + 
                ["Final_Score","Id_Centro", "Centro_Nombre_match", "Centro_Provincia_match", "Similitud_Centro"])

data_3 = data.withColumn("Final_Score", col("Similitud"))\
        .withColumn("Id_Centro", col("Id"))\
        .withColumn("Centro_Nombre_match", col("NOMBRE_ENTIDAD_NIVEL_1"))\
        .withColumn("Centro_Provincia_match", col("COD_PROVINCIA_NIVEL_1"))\
        .withColumn("Similitud_Centro", col("Similitud"))\
        .select(Nombre_Input_investigadores.columns + 
                ["Final_Score","Id_Centro", "Centro_Nombre_match", "Centro_Provincia_match", "Similitud_Centro"])


Nombre_Output_investigadores = data_2.union(data_3)

# =============================================================================
# Aplicación de metodo para almacenar el resultado (parquet, csv, o ambos inclusive)
# =============================================================================
save_csv_parquet(Nombre_Output_investigadores, Flag_csv, Ruta_Nombre_Output_investigadores)


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 45520)
Traceback (most recent call last):
  File "/usr/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.8/socketserver.py", line 720, in __init__
    self.handle()
  File "/home/gaszsantana/virtual/lib/python3.8/site-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/home/gaszsantana/virtual/lib/python3.8/site-packages/pyspark/accumulators.py", line 253, in poll
    if func():
  File "/home/gaszsantana/virtual/lib/python3.8/site-packages/pyspark/accumulators.py", line 257, in accum_updates
    num_u

ConnectionRefusedError: [Errno 111] Connection refused

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/gaszsantana/virtual/lib/python3.8/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/gaszsantana/virtual/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/gaszsantana/virtual/lib/python3.8/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [ ]:
# =============================================================================
# Creación del archivo Nombre_Intermedio_cruce1_relaciones
# =============================================================================
Nombre_Intermedio_cruce1_relaciones = data.select(["Id", "IdOld", "Entidad", "Centro","Centro_Norm", "Provincia_Entidad", "Provincia_Centro","CIF","Entidad_Norm"])

In [ ]:
FuncionAzul(Ruta_Nombre_Input_entidades = "DATA_SIC_ENTIDADES_PUBLICAS_AYUDAS_ESP_short1.csv", 
            Ruta_Nombre_Input_investigadores = "DATA_SIC_AYUDA_ESP_UO_PB_J_MAESTRO_short1.csv",     
            Ruta_Nombre_Input_relaciones = "DATA_RELACION_ENTIDADES_MAESTRO_short1.csv",
            Ruta_Nombre_Input_invente = 'Analisis_entidades_invente_15-02-2022.xlsx', 
            Ruta_Nombre_Input_dir3 = "DIR3_concatenado_AGE_CCAA_EELL_Universidades.csv", 
            Ruta_Nombre_Intermedio_cruce1_relaciones = "test/Nombre_Intermedio_cruce1_relaciones",
            Ruta_Nombre_Intermedio_cruce1_entidades = "test/Nombre_Intermedio_cruce1_entidades", 
            Ruta_Nombre_Intermedio_cruce2_invente = "test/Nombre_Intermedio_cruce2_invente", 
            Ruta_Nombre_Intermedio_cruce3_dir3 = "test/Nombre_Intermedio_cruce3_dir3",
            Ruta_Nombre_Output_investigadores = "test/Nombre_Output_investigadores", 
            Ruta_Nombre_Output_relaciones = "test/Ruta_Nombre_Output_relaciones",
            Ruta_Nombre_Output_entidades = "test/Ruta_Nombre_Output_entidades", 
            flag_invente_dir3 = False, 
            Flag_csv = True)